In [1]:
import requests
import base64

# Replace with your own Client ID and Client Secret
CLIENT_ID = '2cb6e185d6e84e8986d8b1c4d10ac854'
CLIENT_SECRET = '51fcc4dd2a7d40b4b6b0807773803346'

# Base64 encode the client ID and client secret
client_credentials = f"{CLIENT_ID}:{CLIENT_SECRET}"
client_credentials_base64 = base64.b64encode(client_credentials.encode())

# Request the access token
token_url = 'https://accounts.spotify.com/api/token'
headers = {
    'Authorization': f'Basic {client_credentials_base64.decode()}'
}
data = {
    'grant_type': 'client_credentials'
}
response = requests.post(token_url, data=data, headers=headers)

if response.status_code == 200:
    access_token = response.json()['access_token']
    print("Access token obtained successfully.")
else:
    print("Error obtaining access token.")
    exit()

Access token obtained successfully.


In [2]:
pip install spotipy

   ---------------------------------------- 0.0/250.3 kB ? eta -:--:--
   - -------------------------------------- 10.2/250.3 kB ? eta -:--:--
   ------ -------------------------------- 41.0/250.3 kB 667.8 kB/s eta 0:00:01
   ------------ -------------------------- 81.9/250.3 kB 770.8 kB/s eta 0:00:01
   -------------- ------------------------ 92.2/250.3 kB 655.4 kB/s eta 0:00:01
   ------------------ ------------------- 122.9/250.3 kB 658.7 kB/s eta 0:00:01
   ------------------------ ------------- 163.8/250.3 kB 756.6 kB/s eta 0:00:01
   ----------------------------- -------- 194.6/250.3 kB 737.3 kB/s eta 0:00:01
   ---------------------------------- --- 225.3/250.3 kB 726.5 kB/s eta 0:00:01
   -------------------------------------- 250.3/250.3 kB 699.8 kB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [3]:
import pandas as pd
import spotipy
from spotipy.oauth2 import SpotifyOAuth

In [4]:
def get_trending_playlist_data(playlist_id, access_token):
    # Set up Spotipy with the access token
    sp = spotipy.Spotify(auth=access_token)

    # Get the tracks from the playlist
    playlist_tracks = sp.playlist_tracks(playlist_id, fields='items(track(id, name, artists, album(id, name)))')

    # Extract relevant information and store in a list of dictionaries
    music_data = []
    for track_info in playlist_tracks['items']:
        track = track_info['track']
        track_name = track['name']
        artists = ', '.join([artist['name'] for artist in track['artists']])
        album_name = track['album']['name']
        album_id = track['album']['id']
        track_id = track['id']

        # Get audio features for the track
        audio_features = sp.audio_features(track_id)[0] if track_id != 'Not available' else None

        # Get release date of the album
        try:
            album_info = sp.album(album_id) if album_id != 'Not available' else None
            release_date = album_info['release_date'] if album_info else None
        except:
            release_date = None

        # Get popularity of the track
        try:
            track_info = sp.track(track_id) if track_id != 'Not available' else None
            popularity = track_info['popularity'] if track_info else None
        except:
            popularity = None

        # Add additional track information to the track data
        track_data = {
            'Track Name': track_name,
            'Artists': artists,
            'Album Name': album_name,
            'Album ID': album_id,
            'Track ID': track_id,
            'Popularity': popularity,
            'Release Date': release_date,
            'Duration (ms)': audio_features['duration_ms'] if audio_features else None,
            'Explicit': track_info.get('explicit', None),
            'External URLs': track_info.get('external_urls', {}).get('spotify', None),
            'Danceability': audio_features['danceability'] if audio_features else None,
            'Energy': audio_features['energy'] if audio_features else None,
            'Key': audio_features['key'] if audio_features else None,
            'Loudness': audio_features['loudness'] if audio_features else None,
            'Mode': audio_features['mode'] if audio_features else None,
            'Speechiness': audio_features['speechiness'] if audio_features else None,
            'Acousticness': audio_features['acousticness'] if audio_features else None,
            'Instrumentalness': audio_features['instrumentalness'] if audio_features else None,
            'Liveness': audio_features['liveness'] if audio_features else None,
            'Valence': audio_features['valence'] if audio_features else None,
            'Tempo': audio_features['tempo'] if audio_features else None,
            # Add more attributes as needed
        }

        music_data.append(track_data)

    # Create a pandas DataFrame from the list of dictionaries
    df = pd.DataFrame(music_data)

    return df

In [7]:
playlist_id = '354jJvuPwRd9x7bw5irLq7'

# Call the function to get the music data from the playlist and store it in a DataFrame
music_df = get_trending_playlist_data(playlist_id, access_token)

# Display the DataFrame
print(music_df)

                                           Track Name  \
0                Neele Neele Ambar Par - Male Version   
1                                    Yeh Sham Mastani   
2            Saagar Jaisi Aankhonwali - From "Saagar"   
3                               Ek Ajnabee Haseena Se   
4   Kishore Kumar Speaks And O Mere Dil Ke Chain F...   
..                                                ...   
95  Aaj Mere Yaar Ki Shaadi Hai - From "Aadmi Sada...   
96                                Chaoonga Main Tujhe   
97                             Hai apna dil to aawara   
98                   Kabhi Kabhi Mere Dil Mein - Duet   
99    O Mere Dil Ke Chain (From "Mere Jeevan Saathi")   

                             Artists  \
0    Kalyanji-Anandji, Kishore Kumar   
1        Kishore Kumar, R. D. Burman   
2        Kishore Kumar, R. D. Burman   
3        Kishore Kumar, R. D. Burman   
4        Kishore Kumar, R. D. Burman   
..                               ...   
95  Mohammed Rafi, Deven Verma, Rav

In [8]:
print(music_df.isnull().sum())

Track Name          0
Artists             0
Album Name          0
Album ID            0
Track ID            0
Popularity          0
Release Date        0
Duration (ms)       0
Explicit            0
External URLs       0
Danceability        0
Energy              0
Key                 0
Loudness            0
Mode                0
Speechiness         0
Acousticness        0
Instrumentalness    0
Liveness            0
Valence             0
Tempo               0
dtype: int64


In [9]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from datetime import datetime
from sklearn.metrics.pairwise import cosine_similarity

In [10]:
data = music_df

In [11]:
def calculate_weighted_popularity(release_date):
    # Convert the release date to datetime object
    release_date = datetime.strptime(release_date, '%Y-%m-%d')

    # Calculate the time span between release date and today's date
    time_span = datetime.now() - release_date

    # Calculate the weighted popularity score based on time span (e.g., more recent releases have higher weight)
    weight = 1 / (time_span.days + 1)
    return weight

In [12]:
# Normalize the music features using Min-Max scaling
scaler = MinMaxScaler()
music_features = music_df[['Danceability', 'Energy', 'Key', 
                           'Loudness', 'Mode', 'Speechiness', 'Acousticness',
                           'Instrumentalness', 'Liveness', 'Valence', 'Tempo']].values
music_features_scaled = scaler.fit_transform(music_features)

In [13]:
# a function to get content-based recommendations based on music features
def content_based_recommendations(input_song_name, num_recommendations=5):
    if input_song_name not in music_df['Track Name'].values:
        print(f"'{input_song_name}' not found in the dataset. Please enter a valid song name.")
        return

    # Get the index of the input song in the music DataFrame
    input_song_index = music_df[music_df['Track Name'] == input_song_name].index[0]

    # Calculate the similarity scores based on music features (cosine similarity)
    similarity_scores = cosine_similarity([music_features_scaled[input_song_index]], music_features_scaled)

    # Get the indices of the most similar songs
    similar_song_indices = similarity_scores.argsort()[0][::-1][1:num_recommendations + 1]

    # Get the names of the most similar songs based on content-based filtering
    content_based_recommendations = music_df.iloc[similar_song_indices][['Track Name', 'Artists', 'Album Name', 'Release Date', 'Popularity']]

    return content_based_recommendations

In [14]:
# a function to get hybrid recommendations based on weighted popularity
def hybrid_recommendations(input_song_name, num_recommendations=5, alpha=0.5):
    if input_song_name not in music_df['Track Name'].values:
        print(f"'{input_song_name}' not found in the dataset. Please enter a valid song name.")
        return

    # Get content-based recommendations
    content_based_rec = content_based_recommendations(input_song_name, num_recommendations)

    # Get the popularity score of the input song
    popularity_score = music_df.loc[music_df['Track Name'] == input_song_name, 'Popularity'].values[0]

    # Calculate the weighted popularity score
    weighted_popularity_score = popularity_score * calculate_weighted_popularity(music_df.loc[music_df['Track Name'] == input_song_name, 'Release Date'].values[0])

    # Combine content-based and popularity-based recommendations based on weighted popularity
    hybrid_recommendations = content_based_rec
    hybrid_recommendations = hybrid_recommendations.append({
        'Track Name': input_song_name,
        'Artists': music_df.loc[music_df['Track Name'] == input_song_name, 'Artists'].values[0],
        'Album Name': music_df.loc[music_df['Track Name'] == input_song_name, 'Album Name'].values[0],
        'Release Date': music_df.loc[music_df['Track Name'] == input_song_name, 'Release Date'].values[0],
        'Popularity': weighted_popularity_score
    }, ignore_index=True)

    # Sort the hybrid recommendations based on weighted popularity score
    hybrid_recommendations = hybrid_recommendations.sort_values(by='Popularity', ascending=False)

    # Remove the input song from the recommendations
    hybrid_recommendations = hybrid_recommendations[hybrid_recommendations['Track Name'] != input_song_name]


    return hybrid_recommendations

In [15]:
input_song_name = "Yeh Sham Mastani"
recommendations = hybrid_recommendations(input_song_name, num_recommendations=5)
print(f"Hybrid recommended songs for '{input_song_name}':")
print(recommendations)

Hybrid recommended songs for 'Yeh Sham Mastani':
                                          Track Name  \
2                                Mere Sapnon Ki Rani   
4                          Aanewala Pal Janewala Hai   
3              Mere Sapnon Ki Rani (From "Aradhana")   
0  Kishore Kumar Speaks And O Mere Dil Ke Chain F...   
1                           Chalte Chalte Yun Hi Koi   

                       Artists                                     Album Name  \
2                Kishore Kumar                  Kishore Kumar Masti Ki Chakri   
4  Kishore Kumar, R. D. Burman                                        Golmaal   
3                Kishore Kumar                                  Krazy Kishore   
0  Kishore Kumar, R. D. Burman          Legends - Kishore - The Prodigy Vol 3   
1              Lata Mangeshkar  Pakeezah (Original Motion Picture Soundtrack)   

  Release Date  Popularity  
2   2021-10-26        57.0  
4   1979-04-20        47.0  
3   2018-04-24        44.0  
0   1998-04

C:\Users\Rupam M\AppData\Local\Temp\ipykernel_14140\209674984.py:18: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  hybrid_recommendations = hybrid_recommendations.append({
